In [2]:
import os

os.getcwd()
# 0b00 小数据测试
# os.chdir('F:\\reevo\\outputs\\cvrp_aco-aco\\2025-10-26_21-49-40')
# 0b00
# os.chdir('F:\\reevo\\outputs\\cvrp_aco-aco\\2025-10-25_22-25-51')
# 0b10
# os.chdir('F:\\reevo\\outputs\\cvrp_aco-aco\\2025-11-07_02-57-33')
# 0b11
os.chdir('F:\\reevo\\outputs\\cvrp_aco-aco\\2025-11-07_09-04-29')
# 0b01
# os.chdir('F:\\reevo\\outputs\\cvrp_aco-aco\\2025-11-07_03-47-35')



os.getcwd()

os.listdir()[:5]


['.hydra',
 'aaa_reflection_report.md',
 'analysis_log.json',
 'best_code_overall_val_stdout.txt',
 'cvrp_aco-aco.log']

## 折叠

In [3]:
import json

with open('analysis_log.json', 'r') as f:
    log_data = json.load(f)

print(log_data.keys())


dict_keys(['path', 'population', 'strict_reflection_mode', 'reflection'])


In [4]:
path = log_data['path']
path = path + '\\'
print(path)

f:\reevo\outputs\cvrp_aco-aco\2025-11-07_09-04-29\


In [5]:
import re
def get_iter_num_from_path(path):
    match = re.search(r'iter(\d+)', path)
    if match:
        iter_num = int(match.group(1))
        return iter_num

# update iteration number in log
for entry in log_data['population']:
    entry['iteration'] = get_iter_num_from_path(entry['code_path'])

In [6]:
def update_ref_generated_ind():
    for refs in log_data['reflection']:
        if refs['type'] == 'short_term':
            for ref in refs['values']:
                for ind in log_data['population']:
                    if ind['generated_by'] == 'crossover':
                        if ref['parent_1_code_path'] in ind['parent_code_path'] and ref['parent_2_code_path'] in ind['parent_code_path']:
                            ref['generated_ind_code_path'] = ind['code_path']
        if refs['type'] == 'long_term':
            refs['generated_ind_code_path'] = []
            for ind in log_data['population']:
                if ind['iteration'] == refs['iteration']:
                    refs['parent_code_paths'] = ind['parent_code_path']
                    refs['generated_ind_code_path'].append(ind['code_path'])

update_ref_generated_ind()

In [7]:
from pprint import pprint

pprint(log_data['reflection'][0])

{'iteration': 2,
 'type': 'short_term',
 'values': [{'generated_ind_code_path': 'problem_iter2_code0.py',
             'parent_1_code_path': 'problem_iter1_code2.py',
             'parent_2_code_path': 'problem_iter1_code20.py',
             'response': 'Focus on demand compatibility, normalize components, '
                         'and use percentile thresholds.'},
            {'generated_ind_code_path': 'problem_iter2_code1.py',
             'parent_1_code_path': 'problem_iter1_code26.py',
             'parent_2_code_path': 'problem_iter1_code10.py',
             'response': 'Use capacity constraints directly; favor clustered '
                         'neighbors; penalize excess demand edges.'},
            {'generated_ind_code_path': 'problem_iter2_code2.py',
             'parent_1_code_path': 'problem_iter1_code8.py',
             'parent_2_code_path': 'problem_iter1_code17.py',
             'response': 'Prioritize capacity constraints and depot alignment. '
                     

In [8]:
a = log_data['reflection'][1]['values'][0]['origin']

print(a)
print(type(a))

- Try combining various factors to determine how promising it is to select an edge.
- Try sparsifying the matrix by setting unpromising elements to zero.
<class 'str'>


In [9]:
def get_ind_by_code_path(code_path):
    for ind in log_data['population']:
        if ind['code_path'] == code_path:
            return ind
    return None

## 生成报告

In [10]:
# 生成报告

report = ""

# 先找出最好的个体

best_ind = None
best_obj = float('inf')
for ind in log_data['population']:
    if ind['obj'] < best_obj:
        best_obj = ind['obj']
        best_ind = ind
report += f"# Best Individual:\n"
report += f"Code Path: {best_ind['code_path']}\n"
report += f"Objective Value: {best_ind['obj']}\n"

used = set()
iter_rate = []
iter_objs = [[]]
for ind in log_data['population']:
    if get_iter_num_from_path(ind['code_path']) == 1:
        iter_objs[-1].append(ind['obj'])

for refs in log_data['reflection']:
    num_new_inds = 0
    num_better_inds = 0
    iter_objs.append([])
    if refs['type'] == 'short_term':
        # continue
        report += f"\n\n## === iteration {refs['iteration']} ===\n"
        report += f"Short-term Reflections:\n"
        for ref in refs['values']:
            if ref['generated_ind_code_path'] in used:
                continue
            used.add(ref['generated_ind_code_path'])
            report += '### ' + '=' * 10 + "  sep  " + '=' * 10 + "\n"
            # report += '=' * 10 + "  parents  " + '=' * 10 + "\n"
            report += f"Parents:\n"
            report += "code --diff " + f"{path + ref['parent_1_code_path']} {path + ref['parent_2_code_path']}\n".replace('code','response').replace('.py','.txt')
            report += f"new individual path: {ref['generated_ind_code_path']}\n"
            report += f"\nshort-term reflection:\n{ref['response']}\n\n"
            report += 'compare objs:\n'
            report += f'parent1 obj: {get_ind_by_code_path(ref["parent_1_code_path"])["obj"]}\n'
            report += f'parent2 obj: {get_ind_by_code_path(ref["parent_2_code_path"])["obj"]}\n'
            report += f'new ind obj: {get_ind_by_code_path(ref["generated_ind_code_path"])["obj"]}\n\n'
            num_new_inds += 1
            iter_objs[-1].append(get_ind_by_code_path(ref['generated_ind_code_path'])['obj'])
            if get_ind_by_code_path(ref['generated_ind_code_path'])['obj'] < min(
                get_ind_by_code_path(ref['parent_1_code_path'])['obj'],
                get_ind_by_code_path(ref['parent_2_code_path'])['obj']
            ):
                num_better_inds += 1
            better_parent = ref['parent_1_code_path'] if get_ind_by_code_path(ref['parent_1_code_path'])['obj'] < get_ind_by_code_path(ref['parent_2_code_path'])['obj'] else ref['parent_2_code_path']
            report += f'compare new ind with better parent:\n'
            report += "code --diff " + f"{path + better_parent} {path + ref['generated_ind_code_path']}\n".replace('code','response').replace('.py','.txt')
            report += '\n'


    if refs['type'] == 'long_term':
        report += f"\n\n## === iteration {refs['iteration']} ===\n"
        report += f"Long-term Reflections:\n"
        parent_paths = refs['parent_code_paths']
        report += f'''
Origin:
{refs['values'][0]['origin']}

New Long-term Reflection:
{refs['values'][0]['response']}

short-term reflections considered:
{refs['values'][0]['short_term_refs']}
'''
        report += '\n' + '=' * 10 + "  parents path " + '=' * 10 + "\n"
        report += f'parents path: {parent_paths}\n'
        for new_ind_code_path in refs['generated_ind_code_path']:
            if new_ind_code_path in used:
                continue
            used.add(new_ind_code_path)
            report += '### ' + '=' * 10 + "  sep  " + '=' * 10 + "\n"
            report += "code --diff " + f"{path + parent_paths} {path + new_ind_code_path}\n".replace('code','response').replace('.py','.txt')
            report += "\ncompare objs\n"
            report += f'new ind obj: {get_ind_by_code_path(new_ind_code_path)["obj"]}\n'
            report += f'parent obj: {get_ind_by_code_path(parent_paths)["obj"]}\n'
            num_new_inds += 1
            iter_objs[-1].append(get_ind_by_code_path(new_ind_code_path)['obj'])
            if get_ind_by_code_path(new_ind_code_path)['obj'] < get_ind_by_code_path(parent_paths)['obj']:
                num_better_inds += 1
    print(f"Iteration {refs['iteration']}: New Individuals = {num_new_inds}, Better Individuals = {num_better_inds}")
    iter_rate.append(f"Iteration {refs['iteration']}: New Individuals = {num_new_inds}, Better Individuals = {num_better_inds}")

Iteration 2: New Individuals = 29, Better Individuals = 11
Iteration 3: New Individuals = 15, Better Individuals = 1
Iteration 4: New Individuals = 30, Better Individuals = 10
Iteration 5: New Individuals = 15, Better Individuals = 0


In [11]:
# print(report)

In [12]:
# save report to file
with open('aaa_reflection_report.md', 'w') as f:
    f.write(report)

## 查看变量

In [13]:
ind = get_ind_by_code_path('problem_iter1_code0.py')
print(ind)

{'response_id': 0, 'code_path': 'problem_iter1_code0.py', 'obj': 11.84241273994199, 'exec_success': True, 'traceback_msg': '', 'iteration': 1, 'generated_by': 'initial_population'}


## 查看每代生成的更优率

In [14]:
for ir in iter_rate:
    print(ir)

Iteration 2: New Individuals = 29, Better Individuals = 11
Iteration 3: New Individuals = 15, Better Individuals = 1
Iteration 4: New Individuals = 30, Better Individuals = 10
Iteration 5: New Individuals = 15, Better Individuals = 0


In [15]:
# 查看具体的每代的 obj，降低精度并对齐显示
i = 1
for obj in iter_objs:
    formatted_line = ""
    for value in obj:
        if value == float('inf'):
            formatted_line += f'{"inf":>10}'  # 右对齐，宽度10
        else:
            formatted_line += f'{value:10.4f}'  # 右对齐，宽度10，4位小数
    print(f"iter {i}",formatted_line)
    i += 1

iter 1    11.8424   13.5352   23.6011   13.5008   13.3091   22.9526   11.4928   13.1701   13.8110   22.8450   15.2748   13.5822   18.9212   13.2272   22.5744   17.5479   18.7262   20.0287   16.8733   18.4924   14.1580   16.7840   15.0243   16.8590   16.1027   18.7057   20.6519   12.8353   16.7819   19.6993
iter 2    19.0038   12.8219   14.0273   13.4771   11.5759   16.6149   13.9681   20.7588   13.1580   17.5962   18.8211       inf   27.1516   16.1761   13.7138   15.2553   16.9983   19.2115   11.4207   23.9415   22.2574   15.5898   16.7635   14.4610   12.9649   19.8827   14.7031   17.3182   13.0299
iter 3    23.6414   23.7645   26.3369   18.2717   34.0471   17.4467   12.7748   19.9582   10.7716   25.8218   26.2395   26.1767   26.9953   25.9745   26.0975
iter 4    11.5520   19.9677   13.6946   11.7150   11.8301   21.0017   11.7711   13.6498   23.8371   13.9578   13.1125   12.4512   14.3915   25.9416   13.0703   13.4812   25.5576   13.6600   23.0925   13.4968   12.9696   11.5778   24.083

In [16]:
# 求一下每一代的可行解的均值

for obj in iter_objs:
    feasible_objs = [value for value in obj if value != float('inf')]
    if feasible_objs:
        average_obj = sum(feasible_objs) / len(feasible_objs)
        print(f"        可行解目标函数均值: {average_obj:.4f}")

        可行解目标函数均值: 16.7637
        可行解目标函数均值: 16.5237
        可行解目标函数均值: 22.9545
        可行解目标函数均值: 15.5502
        可行解目标函数均值: 17.0113


In [ ]:
# 不同代的最优值的变化
cur_gen = 1
cur_bset = ''
bset_obj_list = []
for ind in log_data['population']:
    if ind['iteration'] != cur_gen:
        print(f"best obj in generate {cur_gen} is: {cur_bset['obj']}")
        bset_obj_list.append(cur_bset['obj'])
        cur_gen += 1
    if cur_bset == '':
        cur_bset = ind
    else:
        if ind['obj'] < cur_bset['obj']:
            cur_bset = ind

best obj in generate 1 is: 11.49277105778333
best obj in generate 2 is: 11.42071195513302
best obj in generate 3 is: 10.771611021511148
best obj in generate 4 is: 10.771611021511148


## 长期反思分析

这一部分是固定的，是对 0b00 大规模实验的分析

#### irer 3

In [18]:
st_refs = '''
Refine demand compatibility; ensure symmetry; improve depot handling.           1
Prioritize depot connectivity, penalize capacity violations, and use adaptive sparsification.
Use capacity-aware demand compatibility and symmetric edge scoring.
Use demand feasibility checks; ensure symmetry; refine angle heuristics.            1
Use angular relationships and per-node sparsity for better routing.
Prioritize depot connections, use adaptive sparsification, and ensure symmetry.             1
Prioritize capacity feasibility and depot connections. Use vectorized operations for efficiency.            1   1
Use capacity constraints directly; prefer depot connections; sparsify per node.
Prioritize depot connectivity and capacity constraints; use percentile sparsification.
Focus on multiplicative scoring, capacity-aware demand pairing, and percentile-based sparsification.            1   1
Focus on depot connectivity and capacity-aware penalties.
Prioritize depot connectivity, use capacity-aware penalties, and sparsify intelligently.
Use continuous metrics; sparsify per node; normalize features.
Use capacity feasibility, demand similarity, and preserve depot connections.            1
Use weighted linear combination; avoid nested loops.
Prioritize capacity constraints and depot connectivity.
Prioritize feasibility, use multiplicative combination, normalize outputs.
Prioritize depot connections, use spatial clustering, apply capacity penalties.
Prioritize capacity feasibility and demand urgency. Use percentile-based sparsification.        1   1
Use angular relationships and weighted component combinations.
Focus on multiplicative scoring, depot proximity, and demand-capacity feasibility.          1   1   1
Focus on capacity feasibility, depot connectivity, and symmetric heuristics.
Use continuous penalties, similarity metrics, and conservative capacity thresholds.
Focus on feasibility, multiplicative combination, and adaptive thresholding.
Prioritize capacity feasibility and demand similarity; use smarter sparsification.          1
Focus on multiplicative scoring, demand-capacity constraints, and local neighborhood analysis.          1
Prioritize depot connectivity, penalize capacity violations, use top-k sparsification.
Prioritize depot connections, balance capacity usage, and refine sparsification.
Multiply heuristics, not add. Use depot proximity. Ensure symmetry.         1
Prioritize capacity feasibility and spatial clustering over demand sums.            1
'''

st_refs = list(filter(lambda x: x.strip() != '', st_refs.split('\n')))

key_words_in_lt_ref = ['multiplicative scoring', 'depot proximity', 'capacity feasibility', 'angular similarity', 'percentile-based', 'sparsif', 'symmetr', 'vectorized operations']

#### iter 5

In [19]:
st_refs = '''
Refine depot connections; adjust capacity normalization; differentiate depot sparsification.
Use stronger capacity penalty; normalize depot proximity differently.
Use additive depot proximity; refine capacity constraints; avoid over-sparsification.
Use multiplicative combination and capacity-aware feasibility scoring.
Use multiplicative combination and node-wise sparsification.
Prioritize feasibility, use polar angles, simplify penalty logic.
Vectorize computations; use angular similarity; prefer smooth capacity penalties.
Prioritize depot connectivity and capacity constraints. Use adaptive sparsification.
Use spatial clustering and angular similarity from depot.
Use multiplicative scoring, angular positioning, and adaptive thresholds.
Use multiplicative combination and sparsification for stronger filtering.
Use multiplicative combination and node-wise sparsification.
Prioritize capacity feasibility and depot connections over complex normalization.
Prioritize capacity constraints; sparsify more aggressively.
Prioritize depot connections. Use per-node sparsification. Balance capacity constraints.
Use angular differences, stronger filtering, and higher percentile thresholds.
Use continuous penalties and weighted additive combination.
Use multiplicative combination and angular similarity from depot.
Use spatial patterns and refined feasibility checks.
Prioritize depot connectivity and capacity constraints over complex normalization.
Use vectorization, cosine similarity, and smoother capacity scaling.
Prioritize capacity constraints; use simpler, consistent sparsification.
Use multiplicative combination and adaptive thresholding.
Use multiplicative combination, capacity-aware routing, and percentile sparsification.
Combine heuristics multiplicatively; use angular similarity; sparsify per-node.
Prioritize depot proximity, angular similarity, and fine-grained capacity feasibility.
Use vectorization, cosine similarity, and depot-specific thresholds.
Focus on depot proximity, angular similarity, and adaptive capacity constraints.
Prioritize capacity feasibility; use continuous scaling over binary thresholds.
Focus on depot proximity, angular similarity, and capacity feasibility. Use multiplicative combination for stronger filtering.
'''
# st_refs 的字母变成小写
st_refs = st_refs.lower()

st_refs = list(filter(lambda x: x.strip() != '', st_refs.split('\n')))

key_words_in_lt_ref = ['capacity feasibility', 'depot connectivity', 'multiplicative scoring', 'angular similarity', 'node-wise', 'percentile', 'sparsif', 'adaptive', 'filtering', 'vectorize computations', 'smoother scaling']

In [20]:
# for x in st_refs:
#     print(x)

#### following

In [21]:

# 将 angular relationships 替代 angular similarity
# key_words_in_lt_ref.append('angular relationships')
# 查看每个关键词在短期反思中出现的次数
for kw in key_words_in_lt_ref:
    count = 0
    for x in st_refs:
        if kw in x:
            count += 1
    print(f'关键词 "{kw}" 在短期反思中出现的次数: {count}')

# 查看每个短期反思被用到长期反思的比例
num = 0
num_used = 0
unused_st_refs = []
for x in st_refs:
    tag = 1
    for kw in key_words_in_lt_ref:
        if kw in x:
            num_used += 1
            tag = 0
            break
    num += 1
    if tag:
        # print(f'未使用关键词的短期反思: {x}')
        unused_st_refs.append(x)
print(f'\n短期反思中包含关键词的比例: {num_used}/{num} = {num_used/num:.2%}\n\n')

for x in unused_st_refs:
    print(x)


'''

# 查看每个关键词在短期反思中出现的次数
for kw in key_words_in_lt_ref:
    count = 0
    for x in st_refs:
        if kw in x:
            count += 1
    print(f'关键词 "{kw}" 在短期反思中出现的次数: {count}')

# 查看每个短期反思被用到长期反思的比例
num = 0
num_used = 0
for x in st_refs:
    for kw in key_words_in_lt_ref:
        if kw in x:
            num_used += 1
            break
    num += 1
print(f'\n短期反思中包含关键词的比例: {num_used}/{num} = {num_used/num:.2%}\n\n')
'''


关键词 "capacity feasibility" 在短期反思中出现的次数: 4
关键词 "depot connectivity" 在短期反思中出现的次数: 2
关键词 "multiplicative scoring" 在短期反思中出现的次数: 1
关键词 "angular similarity" 在短期反思中出现的次数: 7
关键词 "node-wise" 在短期反思中出现的次数: 2
关键词 "percentile" 在短期反思中出现的次数: 2
关键词 "sparsif" 在短期反思中出现的次数: 11
关键词 "adaptive" 在短期反思中出现的次数: 4
关键词 "filtering" 在短期反思中出现的次数: 3
关键词 "vectorize computations" 在短期反思中出现的次数: 1
关键词 "smoother scaling" 在短期反思中出现的次数: 0

短期反思中包含关键词的比例: 23/30 = 76.67%


use stronger capacity penalty; normalize depot proximity differently.
use multiplicative combination and capacity-aware feasibility scoring.
prioritize feasibility, use polar angles, simplify penalty logic.
use continuous penalties and weighted additive combination.
use spatial patterns and refined feasibility checks.
use vectorization, cosine similarity, and smoother capacity scaling.
use vectorization, cosine similarity, and depot-specific thresholds.


'\n\n# 查看每个关键词在短期反思中出现的次数\nfor kw in key_words_in_lt_ref:\n    count = 0\n    for x in st_refs:\n        if kw in x:\n            count += 1\n    print(f\'关键词 "{kw}" 在短期反思中出现的次数: {count}\')\n\n# 查看每个短期反思被用到长期反思的比例\nnum = 0\nnum_used = 0\nfor x in st_refs:\n    for kw in key_words_in_lt_ref:\n        if kw in x:\n            num_used += 1\n            break\n    num += 1\nprint(f\'\n短期反思中包含关键词的比例: {num_used}/{num} = {num_used/num:.2%}\n\n\')\n'

### 关键词重复 分析

In [22]:
keywords_pre = ['multiplicative scoring', 'depot proximity', 'capacity feasibility', 'angular similarity', 'percentile-based', 'sparsif', 'symmetr', 'vectorized operations']
keywords_new = ['capacity feasibility', 'depot connectivity', 'multiplicative scoring', 'angular similarity', 'node-wise', 'percentile', 'sparsif', 'adaptive', 'filtering', 'vectorize computations', 'smoother scaling']

In [23]:
commom_keywords = set(keywords_pre).intersection(set(keywords_new))
print('commom keywords:',commom_keywords)

commom keywords: {'capacity feasibility', 'multiplicative scoring', 'angular similarity', 'sparsif'}


In [24]:
print(len(commom_keywords))
print(len(keywords_pre))
print(len(keywords_new))
print(f"原来有关键词{len(keywords_pre)}个，更新后有{len(keywords_new)}个，重复的关键词有{len(commom_keywords)}个")

4
8
11
原来有关键词8个，更新后有11个，重复的关键词有4个


## 占位